## Synthetic graphs performance
This notebook contains some code that can be used for performing the experiments showed on section 3.2.1 for synthetic graphs


The model is trained and tested over the different sets of synthetic graphs considered

In [7]:
import sys
sys.path.append("./functions")

from utils import *
from model_bet import *
from layer import *

# General synthetic performance

### The different graphs are generated

In [ ]:


param = {
    "min_nodes": 500,#5000,
    "max_nodes": 1000,#10000,
    "num_of_graphs": 15,
    "graph_types": ["ER","SF"],#,"GRP"],
    "generation_seeds": [10]
}

for graph_type in param["graph_types"]:

    for seed in param["generation_seeds"]:
        
        random.seed(seed)
        print(f"Generating {param['num_of_graphs']} {graph_type} graphs")
        list_bet_data = list()
        for i in range(param['num_of_graphs']):
            print(f"{datetime.now().strftime('%d/%m/%Y %H:%M:%S')}: Graph index:{i+1}/{param['num_of_graphs']}")
            g_nx = create_graph(graph_type,param['min_nodes'],param['max_nodes'])
            
            if nx.number_of_isolates(g_nx)>0:
                g_nx.remove_nodes_from(list(nx.isolates(g_nx)))
                g_nx = nx.convert_node_labels_to_integers(g_nx)

            g_nkit = nx2nkit(g_nx)
            bet_dict = cal_exact_bet(g_nkit)
            deg_dict = cal_exact_degree(g_nkit)
            list_bet_data.append([g_nx,bet_dict,deg_dict])

        fname_bet = f"./graphs/{graph_type}_{param['num_of_graphs']}_graphs_{param['max_nodes']}_{param['min_nodes']}_nodes_{seed}_genseed.pickle"    

        with open(fname_bet,"wb") as fopen:
            pickle.dump(list_bet_data,fopen)

print("Graphs saved")

### The corresponding datasets are created (data split)

In [9]:
param = {
    
    "graph_files": ["ER_15_graphs_1000_500_nodes",#["ER_15_graphs_10000_5000_nodes",
                    "SF_15_graphs_1000_500_nodes"],# "SF_15_graphs_10000_5000_nodes",
                    # "GRP_15_graphs_10000_5000_nodes"],
    "generation_seeds": [10],
    
    "num_copies": [10],#[100],
    "split_seeds": [10],
    "adj_size" : 10000,
    "num_train" : 5,
    "num_test" : 10,
    
}


for graph_file in param["graph_files"]:
    for genseed in param["generation_seeds"]:
        for num_copies in param["num_copies"]:
            for splitseed in param["split_seeds"]:
        
                with open(f"./graphs/{graph_file}_{genseed}_genseed.pickle","rb") as fopen:
                    list_data = pickle.load(fopen)

                num_graph = len(list_data)
                assert param["num_train"]+param["num_test"] == num_graph,"Required split size doesn't match number of graphs in pickle file."
            
                #For training split
                if param["num_train"] > 0:
                    random.seed(splitseed)
                    list_graph, list_n_sequence, list_node_num, cent_mat, deg_mat = create_dataset(list_data[:param["num_train"]],num_copies = num_copies, adj_size=param["adj_size"])

                    with open(f"./data_splits/train/{graph_file}_{genseed}_genseed_{param['num_train']}_train_{num_copies}_copies_{param['adj_size']}_size_{splitseed}_splitseed.pickle","wb") as fopen:
                        pickle.dump([list_graph,list_n_sequence,list_node_num,cent_mat, deg_mat],fopen)

                #For test split
                if param["num_test"] > 0:
                    random.seed(splitseed)
                    list_graph, list_n_sequence, list_node_num, cent_mat, deg_mat = create_dataset(list_data[param["num_train"]:param["num_train"]+param["num_test"]],num_copies = 1,adj_size=param["adj_size"])

                    with open(f"./data_splits/test/{graph_file}_{genseed}_genseed_{param['num_test']}_test_{param['adj_size']}_size_{splitseed}_splitseed.pickle","wb") as fopen:
                        pickle.dump([list_graph,list_n_sequence,list_node_num,cent_mat, deg_mat],fopen)


### The model's performance is analysed when training and testing over synthetic graphs

In [ ]:
 
import pandas as pd

param = {
    "graph_files": ["ER_15_graphs_1000_500_nodes",#["ER_15_graphs_10000_5000_nodes",
                    "SF_15_graphs_1000_500_nodes"],# "SF_15_graphs_10000_5000_nodes",
                                                    # "GRP_15_graphs_10000_5000_nodes"],
    "generation_seeds": [10],

    "num_copies": [10],#[100],
    "split_seeds": [10],
    "adj_size" : 10000,
    "num_train" : 5,
    "num_test" : 10,

    "model_seeds": [15],
    "num_epochs": 15,
}

Results = { "gtype_train":[],
            "generation_seed":[],
            "splilt_seed": [],
            "copies":[],
            "adj_size": [],
            "model_seed": [],
            "epochs": [],
            "kendalltau":[],
            "std":[]}


for graph_file in param["graph_files"]:
    for genseed in param["generation_seeds"]:
        for splitseed in param["split_seeds"]:
            
            test_file = f"{graph_file}_{genseed}_genseed_{param['num_test']}_test_{param['adj_size']}_size_{splitseed}_splitseed.pickle"
            #Load test data
            with open("./data_splits/test/"+test_file,"rb") as fopen:
                list_graph_test,list_n_seq_test,list_num_node_test,bc_mat_test,deg_mat_test = pickle.load(fopen)

            list_adj_test,list_adj_t_test = graph_to_adj_bet(list_graph_test,list_n_seq_test,list_num_node_test,param['adj_size'])

            for num_copies in param["num_copies"]:

                train_file = f"{graph_file}_{genseed}_genseed_{param['num_train']}_train_{num_copies}_copies_{param['adj_size']}_size_{splitseed}_splitseed.pickle"
                #Load training data
                print(f"Loading data...")
                with open("./data_splits/train/"+train_file,"rb") as fopen:
                    list_graph_train,list_n_seq_train,list_num_node_train,bc_mat_train,deg_mat_train = pickle.load(fopen)

                model_size = bc_mat_train.shape[0]
                assert model_size == param['adj_size']
                
                list_adj_train,list_adj_t_train = graph_to_adj_bet(list_graph_train,list_n_seq_train,list_num_node_train,param['adj_size'])
                
                for model_seed in param["model_seeds"]:
                    #Model parameters

                    torch.manual_seed(model_seed)
                    hidden = 20
                    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
                    model = GNN_Bet(ninput=model_size,nhid=hidden,dropout=0.6)
                    model.to(device)

                    optimizer = torch.optim.Adam(model.parameters(),lr=0.0005)
                    num_epoch = param["num_epochs"]

                    print(f"Training, total Number of epoches: {num_epoch}")
                    for e in range(num_epoch):
                        print(f"Epoch number: {e+1}/{num_epoch}")
                        train(list_adj_train,list_adj_t_train,list_num_node_train,bc_mat_train,model,device,optimizer,model_size)

                        #to check test loss while training
                        with torch.no_grad():
                            r = test(list_adj_test,list_adj_t_test,list_num_node_test,bc_mat_test,deg_mat_test,model,device,model_size)

                        Results["gtype_train"].append(train_file)
                        Results["generation_seed"].append(genseed)
                        Results["splilt_seed"].append(splitseed)
                        Results["copies"].append(num_copies)
                        Results["adj_size"].append(model_size)
                        Results["model_seed"].append(model_seed)
                        Results["epochs"].append(e)
                        Results["kendalltau"].append(r["kt"])
                        Results["std"].append(r["std"])

                        df = pd.DataFrame.from_dict(Results)
                        #df.to_csv("output_synthetic_graphs_performance.csv")
                        df.to_csv("./outputs/synthetic_graphs_performance_exp1.csv")


# Varying the replication parameter

### Some datasets are created varying replication parameter

In [5]:
param = {
    
    "graph_files": ["ER_15_graphs_1000_500_nodes",#["ER_15_graphs_10000_5000_nodes",
                    "SF_15_graphs_1000_500_nodes"],# "SF_15_graphs_10000_5000_nodes",
                    # "GRP_15_graphs_10000_5000_nodes"],
    "generation_seeds": [10],
    
    "num_copies": [1,2,10,20,40],
    "split_seeds": [10],
    "adj_size" : 10000,
    "num_train" : 5,
    "num_test" : 10,
    
}


for graph_file in param["graph_files"]:
    for genseed in param["generation_seeds"]:

        with open(f"./graphs/{graph_file}_{genseed}_genseed.pickle","rb") as fopen:
            list_data = pickle.load(fopen)

        num_graph = len(list_data)
        assert param["num_train"]+param["num_test"] == num_graph,"Required split size doesn't match number of graphs in pickle file."
    
        for splitseed in param["split_seeds"]:
            
            random.seed(splitseed)

            #For test split
            if param["num_test"] > 0:
                list_graph, list_n_sequence, list_node_num, cent_mat, deg_mat = create_dataset(list_data[param["num_train"]:param["num_train"]+param["num_test"]],num_copies = 1,adj_size=param["adj_size"])

                with open(f"./data_splits/test/{graph_file}_{genseed}_genseed_{param['num_test']}_test_{param['adj_size']}_size_{splitseed}_splitseed.pickle","wb") as fopen:
                    pickle.dump([list_graph,list_n_sequence,list_node_num,cent_mat, deg_mat],fopen)
                    
            for num_copies in param["num_copies"]:
                
                random.seed(splitseed)
                
                #For training split
                if param["num_train"] > 0:
                    
                    list_graph, list_n_sequence, list_node_num, cent_mat, deg_mat = create_dataset(list_data[:param["num_train"]],num_copies = num_copies, adj_size=param["adj_size"])

                    with open(f"./data_splits/train/{graph_file}_{genseed}_genseed_{param['num_train']}_train_{num_copies}_copies_{param['adj_size']}_size_{splitseed}_splitseed.pickle","wb") as fopen:
                        pickle.dump([list_graph,list_n_sequence,list_node_num,cent_mat, deg_mat],fopen)



### The performance of the  model is tested when varying the replication parameter

In [ ]:
 
import pandas as pd

param = {
    "graph_files": ["ER_15_graphs_1000_500_nodes",#["ER_15_graphs_10000_5000_nodes",
                    "SF_15_graphs_1000_500_nodes"],# "SF_15_graphs_10000_5000_nodes",
                                                    # "GRP_15_graphs_10000_5000_nodes"],
    "generation_seeds": [10],

    "num_copies": [1,2,10,20,40],
    "split_seeds": [10],
    "adj_size" : 10000,
    "num_train" : 5,
    "num_test" : 10,

    "model_seeds": [15],
    "num_epochs": 15,
}

Results = { "gtype_train":[],
            "generation_seed":[],
            "splilt_seed": [],
            "copies":[],
            "adj_size": [],
            "model_seed": [],
            "epochs": [],
            "kendalltau":[],
            "std":[]}


for graph_file in param["graph_files"]:
    for genseed in param["generation_seeds"]:
        for splitseed in param["split_seeds"]:
            
            test_file = f"{graph_file}_{genseed}_genseed_{param['num_test']}_test_{param['adj_size']}_size_{splitseed}_splitseed.pickle"
            #Load test data
            with open("./data_splits/test/"+test_file,"rb") as fopen:
                list_graph_test,list_n_seq_test,list_num_node_test,bc_mat_test,deg_mat_test = pickle.load(fopen)

            list_adj_test,list_adj_t_test = graph_to_adj_bet(list_graph_test,list_n_seq_test,list_num_node_test,param['adj_size'])

            for num_copies in param["num_copies"]:

                train_file = f"{graph_file}_{genseed}_genseed_{param['num_train']}_train_{num_copies}_copies_{param['adj_size']}_size_{splitseed}_splitseed.pickle"
                #Load training data
                print(f"Loading data...")
                with open("./data_splits/train/"+train_file,"rb") as fopen:
                    list_graph_train,list_n_seq_train,list_num_node_train,bc_mat_train,deg_mat_train = pickle.load(fopen)

                model_size = bc_mat_train.shape[0]
                assert model_size == param['adj_size']
                
                list_adj_train,list_adj_t_train = graph_to_adj_bet(list_graph_train,list_n_seq_train,list_num_node_train,param['adj_size'])
                
                for model_seed in param["model_seeds"]:
                    #Model parameters

                    torch.manual_seed(model_seed)
                    hidden = 20
                    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
                    model = GNN_Bet(ninput=model_size,nhid=hidden,dropout=0.6)
                    model.to(device)

                    optimizer = torch.optim.Adam(model.parameters(),lr=0.0005)
                    num_epoch = param["num_epochs"]

                    print(f"Training, total Number of epoches: {num_epoch}")
                    for e in range(num_epoch):
                        print(f"Epoch number: {e+1}/{num_epoch}")
                        train(list_adj_train,list_adj_t_train,list_num_node_train,bc_mat_train,model,device,optimizer,model_size)

                        #to check test loss while training
                        with torch.no_grad():
                            r = test(list_adj_test,list_adj_t_test,list_num_node_test,bc_mat_test,deg_mat_test,model,device,model_size)

                        Results["gtype_train"].append(train_file)
                        Results["generation_seed"].append(genseed)
                        Results["splilt_seed"].append(splitseed)
                        Results["copies"].append(num_copies)
                        Results["adj_size"].append(model_size)
                        Results["model_seed"].append(model_seed)
                        Results["epochs"].append(e)
                        Results["kendalltau"].append(r["kt"])
                        Results["std"].append(r["std"])

                        df = pd.DataFrame.from_dict(Results)
                        #df.to_csv("output_synthetic_graphs_performance.csv")
                        df.to_csv("./outputs/synthetic_graphs_performance_varying_replication_parameter.csv")


# Considering variations on replication and graph generation random seeds

### A set of 10 synthetic graphs are created for testing

In [ ]:


param = {
    "min_nodes": 500,#5000,
    "max_nodes": 1000,#10000,
    "num_of_graphs": 10,
    "graph_types": ["ER","SF"],#,"GRP"],
    "generation_seeds": [10]
}

for graph_type in param["graph_types"]:

    for seed in param["generation_seeds"]:
        
        random.seed(seed)
        print(f"Generating {param['num_of_graphs']} {graph_type} graphs")
        list_bet_data = list()
        for i in range(param['num_of_graphs']):
            print(f"{datetime.now().strftime('%d/%m/%Y %H:%M:%S')}: Graph index:{i+1}/{param['num_of_graphs']}")
            g_nx = create_graph(graph_type,param['min_nodes'],param['max_nodes'])
            
            if nx.number_of_isolates(g_nx)>0:
                g_nx.remove_nodes_from(list(nx.isolates(g_nx)))
                g_nx = nx.convert_node_labels_to_integers(g_nx)

            g_nkit = nx2nkit(g_nx)
            bet_dict = cal_exact_bet(g_nkit)
            deg_dict = cal_exact_degree(g_nkit)
            list_bet_data.append([g_nx,bet_dict,deg_dict])

        fname_bet = f"./graphs/{graph_type}_{param['num_of_graphs']}_graphs_{param['max_nodes']}_{param['min_nodes']}_nodes_{seed}_genseed.pickle"    

        with open(fname_bet,"wb") as fopen:
            pickle.dump(list_bet_data,fopen)

print("Graphs saved")

### A set of 5 synthetic training graphs are created using different random seeds

In [ ]:


param = {
    "min_nodes": 500,#5000,
    "max_nodes": 1000,#10000,
    "num_of_graphs": 5,
    "graph_types": ["ER","SF"],#,"GRP"],
    "generation_seeds": [j for j in range(5)]
}

for graph_type in param["graph_types"]:

    for seed in param["generation_seeds"]:
        
        random.seed(seed)
        print(f"Generating {param['num_of_graphs']} {graph_type} graphs")
        list_bet_data = list()
        for i in range(param['num_of_graphs']):
            print(f"{datetime.now().strftime('%d/%m/%Y %H:%M:%S')}: Graph index:{i+1}/{param['num_of_graphs']}")
            g_nx = create_graph(graph_type,param['min_nodes'],param['max_nodes'])
            
            if nx.number_of_isolates(g_nx)>0:
                g_nx.remove_nodes_from(list(nx.isolates(g_nx)))
                g_nx = nx.convert_node_labels_to_integers(g_nx)

            g_nkit = nx2nkit(g_nx)
            bet_dict = cal_exact_bet(g_nkit)
            deg_dict = cal_exact_degree(g_nkit)
            list_bet_data.append([g_nx,bet_dict,deg_dict])

        fname_bet = f"./graphs/{graph_type}_{param['num_of_graphs']}_graphs_{param['max_nodes']}_{param['min_nodes']}_nodes_{seed}_genseed.pickle"    

        with open(fname_bet,"wb") as fopen:
            pickle.dump(list_bet_data,fopen)

print("Graphs saved")

### The splits for the test sets are created

In [16]:
param = {
    
    "graph_files": ["ER_10_graphs_1000_500_nodes",#["ER_15_graphs_10000_5000_nodes",
                    "SF_10_graphs_1000_500_nodes"],# "SF_15_graphs_10000_5000_nodes",
                    # "GRP_15_graphs_10000_5000_nodes"],
    "generation_seeds": [10],
    
    "num_copies": [10],#[100],
    "split_seeds": [0],
    "adj_size" : 10000,
    "num_train" : 0,
    "num_test" : 10,
    
}


for graph_file in param["graph_files"]:
    for genseed in param["generation_seeds"]:
        for num_copies in param["num_copies"]:
            for splitseed in param["split_seeds"]:
        
                with open(f"./graphs/{graph_file}_{genseed}_genseed.pickle","rb") as fopen:
                    list_data = pickle.load(fopen)

                num_graph = len(list_data)
                assert param["num_train"]+param["num_test"] == num_graph,"Required split size doesn't match number of graphs in pickle file."
            
                #For training split
                if param["num_train"] > 0:
                    random.seed(splitseed)
                    list_graph, list_n_sequence, list_node_num, cent_mat, deg_mat = create_dataset(list_data[:param["num_train"]],num_copies = num_copies, adj_size=param["adj_size"])

                    with open(f"./data_splits/train/{graph_file}_{genseed}_genseed_{param['num_train']}_train_{num_copies}_copies_{param['adj_size']}_size_{splitseed}_splitseed.pickle","wb") as fopen:
                        pickle.dump([list_graph,list_n_sequence,list_node_num,cent_mat, deg_mat],fopen)

                #For test split
                if param["num_test"] > 0:
                    random.seed(splitseed)
                    list_graph, list_n_sequence, list_node_num, cent_mat, deg_mat = create_dataset(list_data[param["num_train"]:param["num_train"]+param["num_test"]],num_copies = 1,adj_size=param["adj_size"])

                    with open(f"./data_splits/test/{graph_file}_{genseed}_genseed_{param['num_test']}_test_{param['adj_size']}_size_{splitseed}_splitseed.pickle","wb") as fopen:
                        pickle.dump([list_graph,list_n_sequence,list_node_num,cent_mat, deg_mat],fopen)


### The splits for the training sets are created using different random seeds generation and fixed split seed

In [22]:
param = {
    
    "graph_files": ["ER_5_graphs_1000_500_nodes",#["ER_15_graphs_10000_5000_nodes",
                    "SF_5_graphs_1000_500_nodes"],# "SF_15_graphs_10000_5000_nodes",
                    # "GRP_15_graphs_10000_5000_nodes"],
    "generation_seeds": [j for j in range(5)],
    
    "num_copies": [10],#[100],
    "split_seeds": [0],
    "adj_size" : 10000,
    "num_train" : 5,
    "num_test" : 0,
    
}

for graph_file in param["graph_files"]:
    for genseed in param["generation_seeds"]:
        for num_copies in param["num_copies"]:
            for splitseed in param["split_seeds"]:
        
                with open(f"./graphs/{graph_file}_{genseed}_genseed.pickle","rb") as fopen:
                    list_data = pickle.load(fopen)

                num_graph = len(list_data)
                assert param["num_train"]+param["num_test"] == num_graph,"Required split size doesn't match number of graphs in pickle file."
            
                #For training split
                if param["num_train"] > 0:
                    random.seed(splitseed)
                    list_graph, list_n_sequence, list_node_num, cent_mat, deg_mat = create_dataset(list_data[:param["num_train"]],num_copies = num_copies, adj_size=param["adj_size"])

                    with open(f"./data_splits/train/{graph_file}_{genseed}_genseed_{param['num_train']}_train_{num_copies}_copies_{param['adj_size']}_size_{splitseed}_splitseed.pickle","wb") as fopen:
                        pickle.dump([list_graph,list_n_sequence,list_node_num,cent_mat, deg_mat],fopen)

                #For test split
                if param["num_test"] > 0:
                    random.seed(splitseed)
                    list_graph, list_n_sequence, list_node_num, cent_mat, deg_mat = create_dataset(list_data[param["num_train"]:param["num_train"]+param["num_test"]],num_copies = 1,adj_size=param["adj_size"])

                    with open(f"./data_splits/test/{graph_file}_{genseed}_genseed_{param['num_test']}_test_{param['adj_size']}_size_{splitseed}_splitseed.pickle","wb") as fopen:
                        pickle.dump([list_graph,list_n_sequence,list_node_num,cent_mat, deg_mat],fopen)


### The splits for the training sets are generated with different split seeds and a generation seed fixed

In [23]:
param = {
    
    "graph_files": ["ER_5_graphs_1000_500_nodes",#["ER_15_graphs_10000_5000_nodes",
                    "SF_5_graphs_1000_500_nodes"],# "SF_15_graphs_10000_5000_nodes",
                    # "GRP_15_graphs_10000_5000_nodes"],
    "generation_seeds": [0],
    
    "num_copies": [10],#[100],
    "split_seeds": [j for j in range(5)],
    "adj_size" : 10000,
    "num_train" : 5,
    "num_test" : 0,
    
}

for graph_file in param["graph_files"]:
    for genseed in param["generation_seeds"]:
        for num_copies in param["num_copies"]:
            for splitseed in param["split_seeds"]:
        
                with open(f"./graphs/{graph_file}_{genseed}_genseed.pickle","rb") as fopen:
                    list_data = pickle.load(fopen)

                num_graph = len(list_data)
                assert param["num_train"]+param["num_test"] == num_graph,"Required split size doesn't match number of graphs in pickle file."
            
                #For training split
                if param["num_train"] > 0:
                    random.seed(splitseed)
                    list_graph, list_n_sequence, list_node_num, cent_mat, deg_mat = create_dataset(list_data[:param["num_train"]],num_copies = num_copies, adj_size=param["adj_size"])

                    with open(f"./data_splits/train/{graph_file}_{genseed}_genseed_{param['num_train']}_train_{num_copies}_copies_{param['adj_size']}_size_{splitseed}_splitseed.pickle","wb") as fopen:
                        pickle.dump([list_graph,list_n_sequence,list_node_num,cent_mat, deg_mat],fopen)

                #For test split
                if param["num_test"] > 0:
                    random.seed(splitseed)
                    list_graph, list_n_sequence, list_node_num, cent_mat, deg_mat = create_dataset(list_data[param["num_train"]:param["num_train"]+param["num_test"]],num_copies = 1,adj_size=param["adj_size"])

                    with open(f"./data_splits/test/{graph_file}_{genseed}_genseed_{param['num_test']}_test_{param['adj_size']}_size_{splitseed}_splitseed.pickle","wb") as fopen:
                        pickle.dump([list_graph,list_n_sequence,list_node_num,cent_mat, deg_mat],fopen)


### The model is  trained and tested considering the diferent  training and testing sets created 

In [ ]:
# considering variations on generation seeds

param = {
    "graph_types": ["ER","SF"],#["ER","SF","GRP"],
    "graphs_sizes": "1000_500_nodes",
    "test_generation_seeds": [10],
    "train_generation_seeds": [j for j in range(5)],
    "test_split_seeds": [0],
    "train_split_seeds": [0],
    "num_copies": [10],
    "adj_size" : 10000,
    "num_train" : 5,
    "num_test" : 10,
    "model_seeds": [15],
    "num_epochs": 15,
}

Results = { "gtype_train":[],
            "train_generation_seed": [],
            "train_splilt_seed": [],
            "test_generation_seed": [],
            "test_splilt_seed": [],
            "copies":[],
            "adj_size": [],
            "model_seed": [],
            "epochs": [],
            "kendalltau":[],
            "std":[]}




for graph_type in param["graph_types"]:
    for testgenseed in param["test_generation_seeds"]:
        for testsplitseed in param["test_split_seeds"]:
            
            test_file = f"{graph_type}_{param['num_test']}_graphs_{param['graphs_sizes']}_{testgenseed}_genseed_{param['num_test']}_test_{param['adj_size']}_size_{testsplitseed}_splitseed.pickle"
            #Load test data
            with open("./data_splits/test/"+test_file,"rb") as fopen:
                list_graph_test,list_n_seq_test,list_num_node_test,bc_mat_test,deg_mat_test = pickle.load(fopen)

            list_adj_test,list_adj_t_test = graph_to_adj_bet(list_graph_test,list_n_seq_test,list_num_node_test,param['adj_size'])

            for traingenseed in param["train_generation_seeds"]:
                for trainsplitseed in param["train_split_seeds"]:
                    for num_copies in param["num_copies"]:
                        
                        train_file = f"{graph_type}_{param['num_train']}_graphs_{param['graphs_sizes']}_{traingenseed}_genseed_{param['num_train']}_train_{num_copies}_copies_{param['adj_size']}_size_{trainsplitseed}_splitseed.pickle"

                        #Load training data
                        print(f"Loading data...")
                        with open("./data_splits/train/"+train_file,"rb") as fopen:
                            list_graph_train,list_n_seq_train,list_num_node_train,bc_mat_train,deg_mat_train = pickle.load(fopen)

                        list_adj_train,list_adj_t_train = graph_to_adj_bet(list_graph_train,list_n_seq_train,list_num_node_train,param['adj_size'])

                        model_size = bc_mat_train.shape[0]
                        assert model_size == param['adj_size']
                        
                        for model_seed in param["model_seeds"]:
                            #Model parameters

                            torch.manual_seed(model_seed)
                            
                            hidden = 20
                            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
                            model = GNN_Bet(ninput=model_size,nhid=hidden,dropout=0.6)
                            model.to(device)

                            optimizer = torch.optim.Adam(model.parameters(),lr=0.0005)
                            num_epoch = param["num_epochs"]

                            print(f"Training, total Number of epoches: {num_epoch}")
                            for e in range(num_epoch):
                                print(f"Epoch number: {e+1}/{num_epoch}")
                                train(list_adj_train,list_adj_t_train,list_num_node_train,bc_mat_train,model,device,optimizer,model_size)

                                #to check test loss while training
                                with torch.no_grad():
                                    r = test(list_adj_test,list_adj_t_test,list_num_node_test,bc_mat_test,deg_mat_test,model,device,model_size)

                                Results["gtype_train"].append(train_file)
                                Results["train_generation_seed"].append(traingenseed)
                                Results["train_splilt_seed"].append(trainsplitseed)
                                Results["test_generation_seed"].append(testgenseed)
                                Results["test_splilt_seed"].append(testsplitseed)
                                Results["copies"].append(num_copies)
                                Results["adj_size"].append(model_size)
                                Results["model_seed"].append(model_seed)
                                Results["epochs"].append(e)
                                Results["kendalltau"].append(r["kt"])
                                Results["std"].append(r["std"])

                                df = pd.DataFrame.from_dict(Results)
                                df.to_csv("./outputs/synthetic_graphs_performance_variating_generation_random_seeds.csv")


In [ ]:
# considering variations on replication seeds

param = {
    "graph_types": ["ER","SF"],#["ER","SF","GRP"],
    "graphs_sizes": "1000_500_nodes",
    "test_generation_seeds": [10],
    "train_generation_seeds": [0],
    "test_split_seeds": [0],
    "train_split_seeds": [j for j in range(5)],
    "num_copies": [10],
    "adj_size" : 10000,
    "num_train" : 5,
    "num_test" : 10,
    "model_seeds": [15],
    "num_epochs": 15,
}

Results = { "gtype_train":[],
            "train_generation_seed": [],
            "train_splilt_seed": [],
            "test_generation_seed": [],
            "test_splilt_seed": [],
            "copies":[],
            "adj_size": [],
            "model_seed": [],
            "epochs": [],
            "kendalltau":[],
            "std":[]}




for graph_type in param["graph_types"]:
    for testgenseed in param["test_generation_seeds"]:
        for testsplitseed in param["test_split_seeds"]:
            
            test_file = f"{graph_type}_{param['num_test']}_graphs_{param['graphs_sizes']}_{testgenseed}_genseed_{param['num_test']}_test_{param['adj_size']}_size_{testsplitseed}_splitseed.pickle"
            #Load test data
            with open("./data_splits/test/"+test_file,"rb") as fopen:
                list_graph_test,list_n_seq_test,list_num_node_test,bc_mat_test,deg_mat_test = pickle.load(fopen)

            list_adj_test,list_adj_t_test = graph_to_adj_bet(list_graph_test,list_n_seq_test,list_num_node_test,param['adj_size'])

            for traingenseed in param["train_generation_seeds"]:
                for trainsplitseed in param["train_split_seeds"]:
                    for num_copies in param["num_copies"]:
                        
                        train_file = f"{graph_type}_{param['num_train']}_graphs_{param['graphs_sizes']}_{traingenseed}_genseed_{param['num_train']}_train_{num_copies}_copies_{param['adj_size']}_size_{trainsplitseed}_splitseed.pickle"

                        #Load training data
                        print(f"Loading data...")
                        with open("./data_splits/train/"+train_file,"rb") as fopen:
                            list_graph_train,list_n_seq_train,list_num_node_train,bc_mat_train,deg_mat_train = pickle.load(fopen)

                        list_adj_train,list_adj_t_train = graph_to_adj_bet(list_graph_train,list_n_seq_train,list_num_node_train,param['adj_size'])

                        model_size = bc_mat_train.shape[0]
                        assert model_size == param['adj_size']
                        
                        for model_seed in param["model_seeds"]:
                            #Model parameters

                            torch.manual_seed(model_seed)
                            
                            hidden = 20
                            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
                            model = GNN_Bet(ninput=model_size,nhid=hidden,dropout=0.6)
                            model.to(device)

                            optimizer = torch.optim.Adam(model.parameters(),lr=0.0005)
                            num_epoch = param["num_epochs"]

                            print(f"Training, total Number of epoches: {num_epoch}")
                            for e in range(num_epoch):
                                print(f"Epoch number: {e+1}/{num_epoch}")
                                train(list_adj_train,list_adj_t_train,list_num_node_train,bc_mat_train,model,device,optimizer,model_size)

                                #to check test loss while training
                                with torch.no_grad():
                                    r = test(list_adj_test,list_adj_t_test,list_num_node_test,bc_mat_test,deg_mat_test,model,device,model_size)

                                Results["gtype_train"].append(train_file)
                                Results["train_generation_seed"].append(traingenseed)
                                Results["train_splilt_seed"].append(trainsplitseed)
                                Results["test_generation_seed"].append(testgenseed)
                                Results["test_splilt_seed"].append(testsplitseed)
                                Results["copies"].append(num_copies)
                                Results["adj_size"].append(model_size)
                                Results["model_seed"].append(model_seed)
                                Results["epochs"].append(e)
                                Results["kendalltau"].append(r["kt"])
                                Results["std"].append(r["std"])

                                df = pd.DataFrame.from_dict(Results)
                                df.to_csv("./outputs/synthetic_graphs_performance_variating_replication_random_seeds.csv")
